In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
os.chdir("/mnt/arrakis/sietch/VectaLearn")
sys.path.append("./src")

In [2]:
from server.vecta_learn import patch

patch()

In [6]:
import openai

res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo", 
    stream=True,
    messages=[
        {
            "role": "system",
            "content": "You are JokeBot. An AI that tells jokes in the style of Joe Rogan."
        },
        {
            "role": "user",
            "content": "Tell me a joke."
        }
    ]
)

for chunk in res:
    content = chunk["choices"][0].get("delta", {}).get("content")
    if content is not None:
        print(content, end="")

Alright, here's one for you: 

So, I was smoking some of that good ol' Devil's Lettuce the other day, and I had this epiphany, man. You ever notice how mosquitoes are like the vegans of the insect world? They suck on you for their precious blood, but the second you give them a taste of their own medicine and swat 'em away, they're all like, "Dude, that's not cool, man." I mean, talk about hypocrisy, right? I'm all for peace and love, but if you're gonna snack on my arm, you better be ready for the consequences, bro.

In [2]:
from rich.traceback import install
install(show_locals=True)

<bound method InteractiveShell.excepthook of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x7f7eef174730>>

In [2]:
# src/server/rag/parsing.py
#
# Code modified from LlamaIndex by Jerry Liu

from typing import Any, List, Optional, Sequence, Union

from pydantic import BaseModel, Field

from llama_index.schema import BaseNode, Document, IndexNode, TextNode
from llama_index.node_parser.interface import NodeParser
from llama_index.callbacks.base import CallbackManager
from llama_index.llms.openai import LLM


TABLE_SUMMARY_QUERY = """\
What is this table about? Give a very concise summary (imagine you are adding a \
caption), and also output whether or not the table should be kept. \
"""


class UnstructuredParser(NodeParser):
    """Unstructured element node parser
    
    Splits a document into TextNodes and Index Nodes corresponding to embedded
    objects (e.g. tables, images).
    """
    
    callback_manager: CallbackManager = Field(
        default_factory=CallbackManager, exclude=True
    )
    llm: Optional[LLM] = Field(
        default=None, description="LLM model to use for summarization."
    )
    table_summary_query_str: str = Field(
        default=TABLE_SUMMARY_QUERY,
        description="Query string to use for summarization.",
    )
    
    def __init__(
        self,
        callback_manager: Optional[CallbackManager] = None,
        llm: Optional[Any] = None,
        table_summary_query_str: str = TABLE_SUMMARY_QUERY,
    ) -> None:
        """Initialize UnstructuredParser"""

        try:
            import lxml
            import unstructured
        except ImportError:
            raise ImportError(
                "This node parser requires the `lxml` and `unstructured` packages."
            )
        
        callback_manager = callback_manager or CallbackManager([])
        
        return super().__init__(
            callback_manager=callback_manager,
            llm=llm,
            table_summary_query_str=table_summary_query_str,
        )
    
    def get_nodes_from_documents(
        self,
        documents: Sequence[str],
        show_progress: bool = False,
    ) -> List[BaseNode]:
        """Parse document(s) into nodes.
        
        Parameters
        ----------
        documents : Sequence[str]
            Sequence of documents to parse.
        """
        pass
    

In [ ]:
from llama_index.readers.file.flat_reader import FlatReader
from pathlib import Path
from rich import inspect

reader = FlatReader()
docs_2021 = reader.load_data(Path("data/tesla_2021_10k.htm"))
docs_2020 = reader.load_data(Path("data/tesla_2020_10k.htm"))

In [ ]:
# Patch openai
import openai
import llama_index.llms.openai_utils
import os

def patched_create(original_create):
    def new_create(*args, **kwargs):
        headers = kwargs.get('headers', {})
        headers["Helicone-Auth"] = f"Bearer {os.environ['HELICONE_API_KEY']}"
        kwargs['headers'] = headers
        return original_create(*args, **kwargs)
    return new_create

# Backup original methods
original_completion_create = openai.Completion.create
original_chat_completion_create = openai.ChatCompletion.create

# Patch the methods
openai.Completion.create = patched_create(original_completion_create)
openai.ChatCompletion.create = patched_create(original_chat_completion_create)

# Now when you use llama_index.llms.openai_utils.get_completion_endpoint,
# the headers argument will be included automatically.


In [ ]:
# create UnstructuredElementNodeParser from LLamaIndex
from llama_index.node_parser import UnstructuredElementNodeParser
from rag.llm import SmithOpenAI

node_parser = UnstructuredElementNodeParser(llm=SmithOpenAI())

In [ ]:
import os
import pickle

if not os.path.exists("data/2021_nodes.pkl"):
    raw_nodes_2021 = node_parser.get_nodes_from_documents(docs_2021)
    pickle.dump(raw_nodes_2021, open("data/2021_nodes.pkl", "wb"))
else:
    raw_nodes_2021 = pickle.load(open("data/2021_nodes.pkl", "rb"))

In [ ]:
len(raw_nodes_2021)

354